In [9]:
import re, string
import csv
from datetime import datetime
text_file = sc.textFile('lab5data/*.tess')

#text_file = sc.textFile('lab5data/latin/*.tess') 
#complete latin text to test, long runtime need
print("Number of partitions: {}".format(text_file.getNumPartitions()))

Number of partitions: 2


In [10]:
latin = {};
with open('new_lemmatizer.csv') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=',')
     for row in spamreader:
         latin[row[1]] = row[0]
# save the latin data as map data structure

In [11]:
text_file.take(5)

['<ambrose. ap_david_altera. 1> Fortasse plerosque psalmi titulus offenderit, quem audistis legi, quod uenit ad eum Nathan propheta, cum intrauit ad Betsabee. simul etiam non mediocre scrupulum mouere potuit inperitis euangelii lectio, quae decursa est, in quo aduertistis adulteram Christo oblatam eandemque sine damnatione dimissam. nam profecto si quis ea auribus accipiat otiosis, incentiuum erroris incurrit, cum legit sancti uiri adulterium et adulterae absolutionem, humano propemodum diuinoque lapsus exemplo, quod et homo putauerit adulterium esse faciendum et deus censuerit adulterium non esse damnandum. lubrica igitur ad lapsum uia uel ueniae uel concupiscentiae.']

In [12]:
string.punctuation;
punc = '!"#$%&\'()*+,./:;=?@[\\]^_`{|}~' # let <> became the exception

## Fallowing function is for 3-gram##

In [6]:
def clean_str(x):
    cvsSplitBy = '>';
    transtable = {ord(c): None for c in punc}
    temp = x.translate(transtable);
    spliter = [];
    spliter = temp.split(cvsSplitBy);
    if len(spliter)!=1:
        cleaned_text = spliter[1];
    else:
        cleaned_text = "";
    cleaned_data = cleaned_text.split(' ');
    converted = "";
    i = 0;
    j = 0;
    d = 0;
    for i in range(0,len(cleaned_data)):
        j = i;
        for j in range(i+1,len(cleaned_data)):
#             if len(cleaned_data[i])!=0 and len(cleaned_data[j])!=0:
#                      converted = converted+cleaned_data[i]+','+cleaned_data[j]+str(spliter[0])+'>'+'@';
            d = j;
            for d in range(j+1,len(cleaned_data)):
                cur = sorted([cleaned_data[i],cleaned_data[j],cleaned_data[d]]);
                if cur[0] in latin:
                    cur[0] = latin[cur[0]];
                if cur[1] in latin:
                    cur[1] = latin[cur[1]];
                if cur[2] in latin:
                    cur[2] = latin[cur[2]];    
                if len(cleaned_data[i])!=0 and len(cleaned_data[j])!=0 and len(cleaned_data[d])!=0:
                    converted = converted+cur[0]+','+cur[1]+','+cur[2]+str(spliter[0])+'>'+'@';
    lowercased_str = converted.lower()
    return lowercased_str

In [ ]:
start=datetime.now()
one_RDD = text_file.flatMap(lambda x: clean_str(x).split('@'))
one_RDD = one_RDD.map(lambda x: (x[:x.find('<')],x[x.find('<'):]))
one_RDD = one_RDD.reduceByKey(lambda x,y: x + y)
one_RDD.saveAsTextFile('TextOutput_1')
print (datetime.now()-start)

# Fallowing function is for 2-gram#

In [ ]:
def clean_str_1(x):
    cvsSplitBy = '>';
    transtable = {ord(c): None for c in punc}
    temp = x.translate(transtable);
    spliter = [];
    spliter = temp.split(cvsSplitBy);
    if len(spliter)!=1:
        cleaned_text = spliter[1];
    else:
        cleaned_text = "";
    cleaned_data = cleaned_text.split(' ');
    converted = "";
    i = 0;
    j = 0;
    d = 0;
    for i in range(0,len(cleaned_data)):
        j = i;
        for j in range(i+1,len(cleaned_data)):
            cur = sorted([cleaned_data[i],cleaned_data[j]]);
            if cur[0] in latin:
                cur[0] = latin[cur[0]];
            if cur[1] in latin:
                cur[1] = latin[cur[1]];
            if len(cleaned_data[i])!=0 and len(cleaned_data[j])!=0:
                     converted = converted+cur[0]+','+cur[1]+str(spliter[0])+'>'+'@';
                     

    lowercased_str = converted.lower()
    return lowercased_str

In [7]:
start=datetime.now()
two_RDD = text_file.flatMap(lambda x: clean_str_1(x).split('@'))
two_RDD = two_RDD.map(lambda x: (x[:x.find('<')],x[x.find('<'):]))
two_RDD = two_RDD.reduceByKey(lambda x,y: x + y)
two_RDD.saveAsTextFile('TextOutput')
print (datetime.now()-start)

0:00:00.834302


#  Output#

In [8]:
print("Number of partitions: {}".format(one_RDD.getNumPartitions()))
one_RDD.take(5)

Number of partitions: 2


[('', ''),
 ('erroris,hu虂manas,quo', '<ambrose apdavidaltera 1>'),
 ('exemploque,lubrica,si虂ne', '<ambrose apdavidaltera 1>'),
 ('erroris,intrauit,quem', '<ambrose apdavidaltera 1>'),
 ('otiosis,putauerit,uenit', '<ambrose apdavidaltera 1>')]

In [ ]:
print("Number of partitions: {}".format(two_RDD.getNumPartitions()))
two_RDD.take(5)